In [1]:
import soccerdata as sd

[10/07/25 00:28:52] INFO     No custom team name replacements found. You can configure these in       ]8;id=906027;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=932010;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py#91\91]8;;\
                             C:\Users\Leon\soccerdata\config\teamname_replacements.json.                           

                    INFO     Custom league dict loaded from                                          ]8;id=983052;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=539245;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py#195\195]8;;\
                             C:\Users\Leon\soccerdata\config\league_dict.json.                                     

In [2]:
fbref = sd.FBref(leagues="BRA-Brazil", seasons="2025")

                    INFO     Saving cached data to C:\Users\Leon\soccerdata\data\FBref               ]8;id=722213;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=804470;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#263\263]8;;\

In [3]:
matches_full = fbref.read_schedule()
matches_full = matches_full.reset_index()
matches = matches_full[['week', 'home_team', 'home_xg', 'away_xg', 'away_team']]
matches.head()

                    WARNING  c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\site-package ]8;id=97494;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\warnings.py\warnings.py]8;;\:]8;id=768132;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\warnings.py#112\112]8;;\
                             s\soccerdata\fbref.py:165: FutureWarning: The behavior of DataFrame                   
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

,week,home_team,home_xg,away_xg,away_team
0,1,Cruzeiro,1.2,1.7,Mirassol
1,1,Flamengo,1.8,0.4,Internacional
2,1,Fortaleza,0.9,0.9,Fluminense
3,1,Grêmio,0.8,2.9,Atlético Mineiro
4,1,Juventude,0.7,0.6,Vitória


In [4]:
team_stats_by_week_full = fbref.read_team_match_stats()
team_stats_by_week_full = team_stats_by_week_full.reset_index()
team_stats_by_week = team_stats_by_week_full[['team', 'round', 'xG', 'xGA']]
team_stats_by_week = team_stats_by_week.dropna()
mask = team_stats_by_week['round'].str.contains('Matchweek')
team_stats_by_week = team_stats_by_week[mask].copy()
# Extract the number that immediately follows "Matchweek "
team_stats_by_week['round'] = team_stats_by_week['round'].str.extract(r'Matchweek (\d+)',expand=False).astype(int)
team_stats_by_week.head()

[10/07/25 00:28:58] WARNING  c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\site-package ]8;id=645439;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\warnings.py\warnings.py]8;;\:]8;id=886585;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\warnings.py#112\112]8;;\
                             s\soccerdata\fbref.py:165: FutureWarning: The behavior of DataFrame                   
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

,team,round,xG,xGA
0,Atlético Mineiro,1,2.9,0.8
2,Atlético Mineiro,2,1.4,0.3
4,Atlético Mineiro,3,1.8,1.3
5,Atlético Mineiro,4,0.8,1.7
6,Atlético Mineiro,5,0.8,0.3


In [5]:
import pandas as pd
# --- 1. Prepare Rolling/Expanding Stats in team_stats_by_week ---

# The rolling window size is 3, but the calculation must be *cumulative/expanding*
# for the early weeks (1, 2, 3).

# Step 1a: Sort the data to ensure the calculation is strictly chronological
team_stats_by_week_sorted = team_stats_by_week.sort_values(by=['team', 'round'])

# Step 1b: Calculate the *cumulative* sum and count of xG and xGA for each team,
# up to the *current* round (including the current round).
team_stats_by_week_sorted['xG_cumsum'] = team_stats_by_week_sorted.groupby('team')['xG'].cumsum()
team_stats_by_week_sorted['xGA_cumsum'] = team_stats_by_week_sorted.groupby('team')['xGA'].cumsum()
team_stats_by_week_sorted['game_count'] = team_stats_by_week_sorted.groupby('team')['round'].cumcount() + 1

# Step 1c: Calculate the Expanding Average (Avg of ALL previous games)
# Note: This is NOT the final 3-game average yet, but it handles the early weeks' averaging.
team_stats_by_week_sorted['xG_expanding_avg'] = (
    team_stats_by_week_sorted['xG_cumsum'] / team_stats_by_week_sorted['game_count']
)
team_stats_by_week_sorted['xGA_expanding_avg'] = (
    team_stats_by_week_sorted['xGA_cumsum'] / team_stats_by_week_sorted['game_count']
)


# Step 1d: Calculate the 3-Game Rolling Sum (for rounds >= 3)
# The rolling calculation only works when the window is full (min_periods=3 is the default).
team_stats_by_week_sorted['xG_3_game_sum'] = team_stats_by_week_sorted.groupby('team')['xG'].rolling(window=3, min_periods=1).sum().reset_index(level=0, drop=True)
team_stats_by_week_sorted['xGA_3_game_sum'] = team_stats_by_week_sorted.groupby('team')['xGA'].rolling(window=3, min_periods=1).sum().reset_index(level=0, drop=True)
team_stats_by_week_sorted['game_count_3_window'] = team_stats_by_week_sorted['game_count'].apply(lambda x: min(x, 3))


# Step 1e: Calculate the Final Trailing Average (handles both expanding and 3-game rolling)
# This average is the sum of the last 3 games (or fewer for early weeks) divided by the count of games in the window.
team_stats_by_week_sorted['xG_trailing'] = (
    team_stats_by_week_sorted['xG_3_game_sum'] / team_stats_by_week_sorted['game_count_3_window']
)
team_stats_by_week_sorted['xGA_trailing'] = (
    team_stats_by_week_sorted['xGA_3_game_sum'] / team_stats_by_week_sorted['game_count_3_window']
)

# Step 1f: Shift the data to get the average *BEFORE* the current round (week - 1)
# This is the crucial step to prevent data leakage (look-ahead bias).
team_stats_by_week_sorted['xG_trailing_before'] = team_stats_by_week_sorted.groupby('team')['xG_trailing'].shift(1)
team_stats_by_week_sorted['xGA_trailing_before'] = team_stats_by_week_sorted.groupby('team')['xGA_trailing'].shift(1)


# Step 1g: Handle the special case for Round 1
# Since the 'shift(1)' makes Round 1 NaN, we set the specified default of 1.0.
team_stats_by_week_sorted['xG_trailing_before'] = team_stats_by_week_sorted['xG_trailing_before'].fillna(1.0)
team_stats_by_week_sorted['xGA_trailing_before'] = team_stats_by_week_sorted['xGA_trailing_before'].fillna(1.0)


# Step 1h: Select only the required columns for merging
trailing_stats = team_stats_by_week_sorted[['team', 'round', 'xG_trailing_before', 'xGA_trailing_before']].copy()
# Rename 'round' to 'week' to match the 'matches' DataFrame for merging
trailing_stats.rename(columns={'round': 'week'}, inplace=True)

# --- 2. Merge Trailing Stats into the Matches DataFrame ---

# Step 2a: Merge HOME team stats
matches = pd.merge(
    matches,
    trailing_stats,
    left_on=['home_team', 'week'],
    right_on=['team', 'week'],
    how='left'
)
# Rename the new columns and drop the redundant 'team' column
matches.rename(columns={
    'xG_trailing_before': 'home_xg_trailing',
    'xGA_trailing_before': 'home_xga_trailing'
}, inplace=True)
matches.drop(columns='team', inplace=True)

# Step 2b: Merge AWAY team stats
matches = pd.merge(
    matches,
    trailing_stats,
    left_on=['away_team', 'week'],
    right_on=['team', 'week'],
    how='left'
)
# Rename the new columns and drop the redundant 'team' column
matches.rename(columns={
    'xG_trailing_before': 'away_xg_trailing',
    'xGA_trailing_before': 'away_xga_trailing'
}, inplace=True)
matches.drop(columns='team', inplace=True)
matches = matches.loc[:, ~matches.columns.duplicated()]
matches = matches.dropna()
# --- Final Check ---
matches

,week,home_team,home_xg,away_xg,away_team,home_xg_trailing,home_xga_trailing,away_xg_trailing,away_xga_trailing
0,1,Cruzeiro,1.2,1.7,Mirassol,1.000000,1.000000,1.000000,1.000000
1,1,Flamengo,1.8,0.4,Internacional,1.000000,1.000000,1.000000,1.000000
2,1,Fortaleza,0.9,0.9,Fluminense,1.000000,1.000000,1.000000,1.000000
3,1,Grêmio,0.8,2.9,Atlético Mineiro,1.000000,1.000000,1.000000,1.000000
4,1,Juventude,0.7,0.6,Vitória,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...
260,27,Ceará,1.2,1.5,Santos,1.100000,1.333333,1.700000,0.566667
261,27,Cruzeiro,1.7,2.3,Sport Recife,0.966667,0.966667,0.833333,1.200000
262,27,Juventude,1.7,1.8,Fortaleza,0.433333,1.200000,1.000000,1.466667
263,27,São Paulo,2.0,1.2,Palmeiras,0.933333,1.033333,1.600000,0.800000


In [6]:
matches['xg_difference'] = matches['home_xg'] - matches['away_xg']
matches

,week,home_team,home_xg,away_xg,away_team,home_xg_trailing,home_xga_trailing,away_xg_trailing,away_xga_trailing,xg_difference
0,1,Cruzeiro,1.2,1.7,Mirassol,1.000000,1.000000,1.000000,1.000000,-0.5
1,1,Flamengo,1.8,0.4,Internacional,1.000000,1.000000,1.000000,1.000000,1.4
2,1,Fortaleza,0.9,0.9,Fluminense,1.000000,1.000000,1.000000,1.000000,0.0
3,1,Grêmio,0.8,2.9,Atlético Mineiro,1.000000,1.000000,1.000000,1.000000,-2.1
4,1,Juventude,0.7,0.6,Vitória,1.000000,1.000000,1.000000,1.000000,0.1
...,...,...,...,...,...,...,...,...,...,...
260,27,Ceará,1.2,1.5,Santos,1.100000,1.333333,1.700000,0.566667,-0.3
261,27,Cruzeiro,1.7,2.3,Sport Recife,0.966667,0.966667,0.833333,1.200000,-0.6
262,27,Juventude,1.7,1.8,Fortaleza,0.433333,1.200000,1.000000,1.466667,-0.1
263,27,São Paulo,2.0,1.2,Palmeiras,0.933333,1.033333,1.600000,0.800000,0.8


In [7]:
matches[matches['home_team']=='Cruzeiro']

,week,home_team,home_xg,away_xg,away_team,home_xg_trailing,home_xga_trailing,away_xg_trailing,away_xga_trailing,xg_difference
0,1,Cruzeiro,1.2,1.7,Mirassol,1.000000,1.000000,1.000000,1.000000,-0.5
39,4,Cruzeiro,2.3,0.3,Bahia,0.866667,1.500000,1.766667,0.766667,2.0
55,6,Cruzeiro,0.8,0.8,Vasco da Gama,1.400000,0.800000,0.966667,1.666667,0.0
65,7,Cruzeiro,1.2,0.3,Flamengo,1.300000,0.933333,2.466667,0.366667,0.9
85,9,Cruzeiro,1.4,0.2,Atlético Mineiro,1.133333,0.700000,1.733333,1.066667,1.2
103,11,Cruzeiro,1.3,0.7,Palmeiras,1.700000,0.633333,1.333333,1.000000,0.6
123,13,Cruzeiro,2.6,0.8,Grêmio,1.433333,0.700000,1.633333,0.866667,1.8
136,15,Cruzeiro,2.7,0.4,Juventude,1.366667,0.966667,0.733333,1.433333,2.3
156,17,Cruzeiro,1.3,0.7,Ceará,1.500000,0.700000,0.800000,0.966667,0.6
176,19,Cruzeiro,2.4,1.7,Santos,1.266667,0.600000,1.866667,2.100000,0.7


In [8]:
matches.columns

Index(['week', 'home_team', 'home_xg', 'away_xg', 'away_team',
       'home_xg_trailing', 'home_xga_trailing', 'away_xg_trailing',
       'away_xga_trailing', 'xg_difference'],
      dtype='object')

In [9]:
matches.head(20)

,week,home_team,home_xg,away_xg,away_team,home_xg_trailing,home_xga_trailing,away_xg_trailing,away_xga_trailing,xg_difference
0,1,Cruzeiro,1.2,1.7,Mirassol,1.0,1.0,1.0,1.0,-0.5
1,1,Flamengo,1.8,0.4,Internacional,1.0,1.0,1.0,1.0,1.4
2,1,Fortaleza,0.9,0.9,Fluminense,1.0,1.0,1.0,1.0,0.0
3,1,Grêmio,0.8,2.9,Atlético Mineiro,1.0,1.0,1.0,1.0,-2.1
4,1,Juventude,0.7,0.6,Vitória,1.0,1.0,1.0,1.0,0.1
5,1,São Paulo,2.4,0.4,Sport Recife,1.0,1.0,1.0,1.0,2.0
6,1,Bahia,1.4,0.5,Corinthians,1.0,1.0,1.0,1.0,0.9
7,1,Palmeiras,1.3,0.9,Botafogo (RJ),1.0,1.0,1.0,1.0,0.4
8,1,Vasco da Gama,1.3,2.0,Santos,1.0,1.0,1.0,1.0,-0.7
9,1,RB Bragantino,1.2,0.9,Ceará,1.0,1.0,1.0,1.0,0.3


**Now tha the dataframe is ready, we can start the creation of the model**

In [ ]:
import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# 1. Load the matches DataFrame
df = matches

# 2. Define new features and target
features = [
    'home_team', 'away_team',
    'home_xg_trailing', 'home_xga_trailing',
    'away_xg_trailing', 'away_xga_trailing'
]
target = 'xg_difference'

X = df[features]
y = df[target]

# 3. Categorical and numerical features
categorical_features = ['home_team', 'away_team']
numerical_features = list(set(features) - set(categorical_features))

# 4. Preprocessor
preprocessor = ColumnTransformer([
    ('cat', ce.TargetEncoder(), categorical_features)
], remainder='passthrough')

# 5. Define pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(objective='reg:squarederror',
    random_state=42,
    tree_method='gpu_hist',
    predictor='gpu_predictor'))
])

# 6. Detailed hyperparameter grid
param_grid = {
    'regressor__n_estimators': [100, 200, 300, 400, 500, 600, 800, 1000],
    'regressor__max_depth': [3, 5, 7, 9, 11, 13, 15, 17],
    'regressor__learning_rate': [
        0.001, 0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3
    ],
    'regressor__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'regressor__colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'regressor__gamma': [0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 1.0],
    'regressor__reg_alpha': [0, 0.001, 0.01, 0.05, 0.1, 1, 10, 50, 100],
    'regressor__reg_lambda': [0, 0.001, 0.01, 0.05, 0.1, 1, 10, 50, 100],
    'regressor__min_child_weight': [1, 3, 5, 7, 10],
    'regressor__scale_pos_weight': [1, 5, 10, 20, 50],
    # Optional: add tree method for further experimentation
    'regressor__tree_method': ['auto', 'exact', 'hist', 'approx']
}


# 7. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 8. Randomized Search CV
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=500,
    cv=5,
    verbose=2,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    random_state=42
)

# 9. Fit the search
search.fit(X_train, y_train)

# 10. Results
print("\n✅ Best parameters found:", search.best_params_)
print("✅ Best CV score (neg MSE):", search.best_score_)

# 11. Evaluate on test set
best_model = search.best_estimator_
y_pred = best_model.predict(X_test)

print("📉 Test Set MSE:", mean_squared_error(y_test, y_pred))
print("📉 Test Set RMSE:", mean_squared_error(y_test, y_pred) ** 0.5)
print("📈 Test Set R²:", r2_score(y_test, y_pred))

# 12. Save model
joblib.dump(best_model, 'xgb_xg_diff_with_teams.pkl')
print("💾 Model saved as 'xgb_xg_diff_with_teams.pkl'")


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[10/07/25 00:32:32] WARNING  c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\site-package ]8;id=415411;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\warnings.py\warnings.py]8;;\:]8;id=540309;file://c:\Users\Leon\AppData\Local\Programs\Python\Python312\Lib\warnings.py#112\112]8;;\
                             s\xgboost\training.py:183: UserWarning: [00:32:32] WARNING:                           
                             C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738:                           
                             Parameters: { "predictor" } are not used.                                             
                                                                                                                   
                               bst.update(dtrain, iteration=i, fobj=obj)                                           
                                                                                                                   


✅ Best parameters found: {'regressor__tree_method': 'approx', 'regressor__subsample': 0.8, 'regressor__scale_pos_weight': 1, 'regressor__reg_lambda': 0.001, 'regressor__reg_alpha': 10, 'regressor__n_estimators': 800, 'regressor__min_child_weight': 7, 'regressor__max_depth': 7, 'regressor__learning_rate': 0.025, 'regressor__gamma': 1.0, 'regressor__colsample_bytree': 0.7}
✅ Best CV score (neg MSE): -0.9587922243296699
📉 Test Set MSE: 0.9683353188685988
📉 Test Set RMSE: 0.9840403034777584
📈 Test Set R²: 0.1353984047457123
💾 Model saved as 'xgb_xg_diff_with_teams.pkl'


In [ ]:
import numpy as np
accuracy = np.mean(np.sign(y_test) == np.sign(y_pred))
print("Acurácia do sinal:", accuracy)

Acurácia do sinal: 0.660377358490566
